In [145]:
import fitz
from df2gspread import df2gspread as d2g
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pandas.io.json import json_normalize
import sys
import re
test=True

In [146]:
%ls

In [147]:
### READ IN PDF
annotated_doc = fitz.open("Draft3-Full-annotated.pdf")
original_doc = fitz.open("LDC-Draft1.pdf")

In [148]:
annotated_doc.save(annotated_doc.name, incremental=True, encryption=0)

In [149]:
## Connect to Google Sheet
## Connect to our service account
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./Jupyter_and_Google_Sheets.json', scope)
gc = gspread.authorize(credentials)
spreadsheet_key = '1IGMIjolRXo8zhPdFYkoma7BGTgZRekOZZnlFDKBY7Hs'

In [150]:
toc = original_doc.getToC(simple=False)
toc

[[1,
  'Cover',
  1,
  {'kind': 1, 'page': 0, 'to': Point(0.0, 0.0), 'xref': 0, 'zoom': 0.0}],
 [1,
  'Table of Contents',
  2,
  {'kind': 1, 'page': 1, 'to': Point(0.0, 0.0), 'xref': 0, 'zoom': 0.0}],
 [1,
  'Chapter 23-1: Introduction',
  13,
  {'kind': 1, 'page': 12, 'to': Point(0.0, 792.0), 'xref': 0, 'zoom': 0.0}],
 [2,
  'Article 23-1A: General Provisions',
  17,
  {'kind': 1, 'page': 16, 'to': Point(0.0, 792.0), 'xref': 0, 'zoom': 0.0}],
 [3,
  'Division 23-1A-1: Title, Purpose, and Scope',
  19,
  {'kind': 1, 'page': 18, 'to': Point(0.0, 792.0), 'xref': 0, 'zoom': 0.0}],
 [3,
  'Division 23-1A-2: Authority',
  23,
  {'kind': 1, 'page': 22, 'to': Point(0.0, 792.0), 'xref': 0, 'zoom': 0.0}],
 [3,
  'Division 23-1A-3: Classification of Applications and Decisions',
  25,
  {'kind': 1, 'page': 24, 'to': Point(0.0, 792.0), 'xref': 0, 'zoom': 0.0}],
 [3,
  'Division 23-1A-4: Consistency with Comprehensive Plan',
  29,
  {'kind': 1, 'page': 28, 'to': Point(0.0, 792.0), 'xref': 0, 'zoom

In [151]:
def find_last_page(row):
    """
    Finds the last page of the given item
    """
    index = int(row.name)
    search_df = tocdf.loc[index:]
    search_df = search_df[search_df['level-number'] <= row['level-number']]
    if search_df.shape[0] > 1:
        return search_df.iloc[1].Page
    else:
        return original_doc.pageCount

#page_count(tocdf.iloc[0], 0, tocdf)

In [152]:
tocdf = pd.DataFrame.from_records(toc, columns=["Level", "Title", "Page", "Destination Dict"])
tocdf = pd.concat([tocdf, pd.DataFrame(tocdf["Destination Dict"].tolist())], axis=1)
tocdf.head()

,Level,Title,Page,Destination Dict,kind,page,to,xref,zoom
0,1,Cover,1,"{u'to': [0.0, 0.0], u'kind': 1, u'xref': 0, u'...",1,0,"[0.0, 0.0]",0,0.0
1,1,Table of Contents,2,"{u'to': [0.0, 0.0], u'kind': 1, u'xref': 0, u'...",1,1,"[0.0, 0.0]",0,0.0
2,1,Chapter 23-1: Introduction,13,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,12,"[0.0, 792.0]",0,0.0
3,2,Article 23-1A: General Provisions,17,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,16,"[0.0, 792.0]",0,0.0
4,3,"Division 23-1A-1: Title, Purpose, and Scope",19,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,18,"[0.0, 792.0]",0,0.0


In [153]:
# chapter_regex = re.compile('_(23-\d+)_')
# article_regex = re.compile('(Article 23-\d+\w):')
# division_regex = re.compile('(Division 23-\d\w-\d):')
# section_regex = re.compile('(23-\d+\w-\d\d\d\d)\s')
all_regex = re.compile('23-(\d+)([A-Z])?-?(\d+)?(\d+)?:?_?\s?\t?')

def title_to_item(title):
#     chapter_match = chapter_regex.search(title)
#     article_match = article_regex.search(title)
#     division_match = division_regex.search(title)
#     section_match = section_regex.search(title)
    all_match = all_regex.search(title)
    if all_match != None:
    
        chapter = all_match.group(1)
        article = all_match.group(2)
        division = all_match.group(3)
        section = all_match.group(4)
        name = title[all_match.end():]

        if section:
            return {"level": "section", "level-number":4, "name":name, "chapter": int(chapter), "article": article, "division": division, "section": section}
        elif division:
            return {"level": "division", "level-number":3, "name":name, "chapter": int(chapter), "article": article, "division": division}
        elif article:
            return {"level": "article", "level-number":2, "name":name, "chapter": int(chapter), "article": article}
        elif chapter:
            return {"level": "chapter", "level-number":1, "name":name, "chapter": int(chapter)}
    else:
        return {"level": "NA", "level-number":0, "name":title, "chapter": 0, "article": 0, "division": 0, "section": 0}
    
#     if chapter_match != None:
#         name = title[chapter_match.end():]
#         chapter = chapter_match.group(1)
#         return {"chapter": chapter, "name": name}
    
#     else if division_match != None:
#         group1 = chapter_match.group(1)
#         name = title[division_match.end():]
#         chapter = group1[10:14]
#         chapter = group1[10:14]
        
#         return {"chapter": chapter, "name": name}
    
#     else if article_match != None:
#         name = title[chapter_match.end():]
#         chapter = chapter_match.group(1)
#         return {"chapter": chapter, "name": name}
    
#     else if section_match != None:
#         name = title[chapter_match.end():]
#         chapter = chapter_match.group(1)
#         return {"chapter": chapter, "name": name}


In [154]:
items = tocdf["Title"].apply(title_to_item)
tocdf = pd.concat([tocdf,  pd.DataFrame(items.tolist())], axis=1)
tocdf.head()

,Level,Title,Page,Destination Dict,kind,page,to,xref,zoom,article,chapter,division,level,level-number,name,section
0,1,Cover,1,"{u'to': [0.0, 0.0], u'kind': 1, u'xref': 0, u'...",1,0,"[0.0, 0.0]",0,0.0,0,0,0,NA,0,Cover,0.0
1,1,Table of Contents,2,"{u'to': [0.0, 0.0], u'kind': 1, u'xref': 0, u'...",1,1,"[0.0, 0.0]",0,0.0,0,0,0,NA,0,Table of Contents,0.0
2,1,Chapter 23-1: Introduction,13,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,12,"[0.0, 792.0]",0,0.0,NaN,1,NaN,chapter,1,Introduction,NaN
3,2,Article 23-1A: General Provisions,17,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,16,"[0.0, 792.0]",0,0.0,A,1,NaN,article,2,General Provisions,NaN
4,3,"Division 23-1A-1: Title, Purpose, and Scope",19,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,18,"[0.0, 792.0]",0,0.0,A,1,1,division,3,"Title, Purpose, and Scope",NaN


In [155]:
tocdf = tocdf[tocdf["level"] != "NA"]
tocdf.head()

,Level,Title,Page,Destination Dict,kind,page,to,xref,zoom,article,chapter,division,level,level-number,name,section
2,1,Chapter 23-1: Introduction,13,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,12,"[0.0, 792.0]",0,0.0,NaN,1,NaN,chapter,1,Introduction,NaN
3,2,Article 23-1A: General Provisions,17,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,16,"[0.0, 792.0]",0,0.0,A,1,NaN,article,2,General Provisions,NaN
4,3,"Division 23-1A-1: Title, Purpose, and Scope",19,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,18,"[0.0, 792.0]",0,0.0,A,1,1,division,3,"Title, Purpose, and Scope",NaN
5,3,Division 23-1A-2: Authority,23,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,22,"[0.0, 792.0]",0,0.0,A,1,2,division,3,Authority,NaN
6,3,Division 23-1A-3: Classification of Applicatio...,25,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,24,"[0.0, 792.0]",0,0.0,A,1,3,division,3,Classification of Applications and Decisions,NaN


In [156]:
def generate_key(row):
    key = ""
    if pd.notnull(row["chapter"]):
        key += "23-" + str(row["chapter"])
    if pd.notnull(row["article"]):
        key += str(row["article"])
    if pd.notnull(row["division"]):
        key += "-" + str(row["division"])
    return key
    

In [157]:
tocdf["Level Number"] = tocdf.apply(lambda x: x[x["level"]], axis=1)
tocdf["Key"] = tocdf.apply(lambda x: generate_key(x), axis=1)
tocdf

,Level,Title,Page,Destination Dict,kind,page,to,xref,zoom,article,chapter,division,level,level-number,name,section,Level Number,Key
2,1,Chapter 23-1: Introduction,13,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,12,"[0.0, 792.0]",0,0.0,NaN,1,NaN,chapter,1,Introduction,NaN,1,23-1
3,2,Article 23-1A: General Provisions,17,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,16,"[0.0, 792.0]",0,0.0,A,1,NaN,article,2,General Provisions,NaN,A,23-1A
4,3,"Division 23-1A-1: Title, Purpose, and Scope",19,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,18,"[0.0, 792.0]",0,0.0,A,1,1,division,3,"Title, Purpose, and Scope",NaN,1,23-1A-1
5,3,Division 23-1A-2: Authority,23,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,22,"[0.0, 792.0]",0,0.0,A,1,2,division,3,Authority,NaN,2,23-1A-2
6,3,Division 23-1A-3: Classification of Applicatio...,25,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,24,"[0.0, 792.0]",0,0.0,A,1,3,division,3,Classification of Applications and Decisions,NaN,3,23-1A-3
7,3,Division 23-1A-4: Consistency with Comprehensi...,29,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,28,"[0.0, 792.0]",0,0.0,A,1,4,division,3,Consistency with Comprehensive Plan,NaN,4,23-1A-4
8,3,Division 23-1A-5: Rules of Interpretation,31,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,30,"[0.0, 792.0]",0,0.0,A,1,5,division,3,Rules of Interpretation,NaN,5,23-1A-5
9,2,Article 23-1B: Responsibility for Administration,33,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,32,"[0.0, 792.0]",0,0.0,B,1,NaN,article,2,Responsibility for Administration,NaN,B,23-1B
10,3,Division 23-1B-1: City Council,35,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,34,"[0.0, 792.0]",0,0.0,B,1,1,division,3,City Council,NaN,1,23-1B-1
11,3,Division 23-1B-2: Boards and Commissions,37,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,36,"[0.0, 792.0]",0,0.0,B,1,2,division,3,Boards and Commissions,NaN,2,23-1B-2


In [158]:
tocdf = tocdf.sort_values(by=['Page','chapter', 'article', 'division', 'section'], na_position='first')
tocdf = tocdf.drop_duplicates(subset=['chapter', 'article', 'division', 'section'], keep='last')
tocdf = tocdf.reset_index(drop=True)


tocdf["link"] = tocdf["Page"].apply(lambda x: "https://ldc.aura-atx.org/pdf/plain-code.pdf#page="+str(x))

tocdf["Last Page"] = tocdf.apply(find_last_page, axis=1)
tocdf["Page Count"] = tocdf["Last Page"] - tocdf["Page"]

#tocdf = tocdf[[u'level', u'chapter', u'article', u'division', u'section', u'name', u'Page', u'Last Page', u'Page Count', u'link']]
tocdf.to_csv("toc.csv")
print(tocdf.shape)
tocdf

,Level,Title,Page,Destination Dict,kind,page,to,xref,zoom,article,...,division,level,level-number,name,section,Level Number,Key,link,Last Page,Page Count
0,3,Division 23-6E-5: Appeal,1,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,0,"[0.0, 792.0]",0,0.0,E,...,5,division,3,Appeal,NaN,5,23-6E-5,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,1,0
1,3,Division 23-7D-1: Enforcement,1,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,0,"[0.0, 792.0]",0,0.0,D,...,1,division,3,Enforcement,NaN,1,23-7D-1,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,1,0
2,3,Division 23-9E-5: Responsibilities of Applican...,1,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,0,"[0.0, 792.0]",0,0.0,E,...,5,division,3,Responsibilities of Applicant or Owner,NaN,5,23-9E-5,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,1,0
3,3,Division 23-9F-1: General Provisions,1,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,0,"[0.0, 792.0]",0,0.0,F,...,1,division,3,General Provisions,NaN,1,23-9F-1,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,1,0
4,3,Division 23-10B-6: Plumbing Code,1,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,0,"[0.0, 792.0]",0,0.0,B,...,6,division,3,Plumbing Code,NaN,6,23-10B-6,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,13,12
5,1,Chapter 23-1: Introduction,13,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,12,"[0.0, 792.0]",0,0.0,NaN,...,NaN,chapter,1,Introduction,NaN,1,23-1,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,51,38
6,2,Article 23-1A: General Provisions,17,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,16,"[0.0, 792.0]",0,0.0,A,...,NaN,article,2,General Provisions,NaN,A,23-1A,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,33,16
7,3,"Division 23-1A-1: Title, Purpose, and Scope",19,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,18,"[0.0, 792.0]",0,0.0,A,...,1,division,3,"Title, Purpose, and Scope",NaN,1,23-1A-1,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,23,4
8,3,Division 23-1A-2: Authority,23,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,22,"[0.0, 792.0]",0,0.0,A,...,2,division,3,Authority,NaN,2,23-1A-2,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,25,2
9,3,Division 23-1A-3: Classification of Applicatio...,25,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,24,"[0.0, 792.0]",0,0.0,A,...,3,division,3,Classification of Applications and Decisions,NaN,3,23-1A-3,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,29,4


In [159]:
tocdf.columns

Index([u'Level', u'Title', u'Page', u'Destination Dict', u'kind', u'page',
       u'to', u'xref', u'zoom', u'article', u'chapter', u'division', u'level',
       u'level-number', u'name', u'section', u'Level Number', u'Key', u'link',
       u'Last Page', u'Page Count'],
      dtype='object')

In [165]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

output_df = tocdf[["Level", "level", "Level Number", "Title",  "Key", "Page", "name", "chapter", "article","division","link","Last Page","Page Count"]]
output_df = output_df.rename(columns={"Level":"Level Index", "level":"Level Name", "Level Number":"Level Value", })
#tocdf["Title"] = tocdf["Title"].apply(is_not_ascii)
d2g.upload(output_df, "1f7ZUpDVnW-FTdBrRL9D9qFgPZfFB-HrnBuiCQnZ2e1U", "table of contents breakdown", credentials=credentials, row_names=True)

reload(sys)
sys.setdefaultencoding("ASCII")

In [161]:
output_df.head()

,Level,Title,Page,Destination Dict,kind,page,to,xref,zoom,article,...,division,level,level-number,name,section,Level Number,Key,link,Last Page,Page Count
0,3,Division 23-6E-5: Appeal,1,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,0,"[0.0, 792.0]",0,0.0,E,...,5,division,3,Appeal,NaN,5,23-6E-5,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,1,0
1,3,Division 23-7D-1: Enforcement,1,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,0,"[0.0, 792.0]",0,0.0,D,...,1,division,3,Enforcement,NaN,1,23-7D-1,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,1,0
2,3,Division 23-9E-5: Responsibilities of Applican...,1,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,0,"[0.0, 792.0]",0,0.0,E,...,5,division,3,Responsibilities of Applicant or Owner,NaN,5,23-9E-5,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,1,0
3,3,Division 23-9F-1: General Provisions,1,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,0,"[0.0, 792.0]",0,0.0,F,...,1,division,3,General Provisions,NaN,1,23-9F-1,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,1,0
4,3,Division 23-10B-6: Plumbing Code,1,"{u'to': [0.0, 792.0], u'kind': 1, u'xref': 0, ...",1,0,"[0.0, 792.0]",0,0.0,B,...,6,division,3,Plumbing Code,NaN,6,23-10B-6,https://ldc.aura-atx.org/pdf/plain-code.pdf#pa...,13,12


In [162]:
original_doc.search("Chapter 23-1: Introduction")

AttributeError: 'Document' object has no attribute 'search'

In [ ]:
     
all_text = []
for page in original_doc: 
    text = page.getText().encode("utf8")
    all_text += text.split()
all_text

In [ ]:
len(all_text)

In [ ]:
df.head()

In [ ]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
import pandas as pd 
  
# Reads 'Youtube04-Eminem.csv' file  
df = pd.read_csv(r"Youtube04-Eminem.csv", encoding ="latin-1")
comment_words = ' '
stopwords = set(STOPWORDS) 
  
# iterate through the csv file 
for val in all_text: 
      
    # typecaste each val to string 
    val = str(val) 
  
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
          
    for words in tokens: 
        comment_words = comment_words + words + ' '
  
  
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

reload(sys)
sys.setdefaultencoding("ASCII")